This is a vectorized version of the environment. How does it work? Simply put, instead of running a single environment, we're running them into batches (vectorization). At each time step, instead of formulating a single action $a$, we'll define it as a vector $a=[0, .., n_\text{envs}]$ where each entry corresponds to the action to be performed for an environment.

Thus, if you run 4 simulatenous environments, your observations space becomes (4, num_observations). Because we use a step size of 10 and have a total of 9 snesors, each observation will result into a space of (40,9) elements. The core idea is to speed up the inference and training time of the model instead of querying a single environment.

In our implementation, based on what model you decide, you should take as inputs the observations reshaped to your liking, and predict the actions $a$ from your policy.

In [1]:
%load_ext autoreload
%autoreload 2

In [61]:
import numpy as np
from student_client.student_gym_env_vectorized import create_student_gym_env_vectorized

env = create_student_gym_env_vectorized(
            num_envs=4,
            user_token='your_token'
        )

2026-02-18 15:00:35,695 - httpx - INFO - HTTP Request: GET http://localhost:8001/api/v1/version "HTTP/1.1 200 OK"
2026-02-18 15:00:35,695 - student_client.student_gym_env_vectorized - INFO - Client version 0.2 is newer than latest 0.1
2026-02-18 15:00:35,696 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/session/create "HTTP/1.1 200 OK"
2026-02-18 15:00:35,697 - student_client.student_gym_env_vectorized - INFO - Created new session: 975afd8d-1e83-4695-b39b-d9060187acf0
2026-02-18 15:00:39,394 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/vectorized/episodes/create "HTTP/1.1 200 OK"
2026-02-18 15:00:39,395 - student_client.student_gym_env_vectorized - INFO - Created vectorized group with 4 episodes
2026-02-18 15:00:39,395 - student_client.student_gym_env_vectorized - INFO - Vectorized group ID: 1ab97b71-385f-4f4b-bb80-c609ee05c9d4
2026-02-18 15:00:39,395 - student_client.student_gym_env_vectorized - INFO - StudentGymEnvVectorized initialized with 4 e

In [56]:
print(f"Environment created with {env.num_envs} parallel environments")
print(f"   Episode IDs: {env.episode_ids}")

# Reset all environments
print(f"\n🔄 Resetting all environments...")
observations, infos = env.reset()

print(f"   Observations shape: {observations.shape}")
print(f"   First observation: {observations[0]}")

Environment created with 4 parallel environments
   Episode IDs: ['35036e75-d5e6-4259-9ddf-436ff9a3f768', '95213856-7fb8-45e1-81eb-e4e8623b23fb', '3c4dd096-a106-4b0d-bae8-b87bb0a62a14', '462b8bc4-c064-489d-a3eb-e622f7920b0e']

🔄 Resetting all environments...


2026-02-18 14:59:00,992 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"
2026-02-18 14:59:00,993 - student_client.student_gym_env_vectorized - INFO - All 4 environments reset successfully


   Observations shape: (4, 9)
   First observation: [7.9793073e+02 1.9422307e+04 3.3630234e+02 1.1223228e+03 3.7253836e-01
 1.3661358e+06 3.9605825e+03 0.0000000e+00 1.0230342e+01]


## Training / Iterations

Here you can iterate through the vectorized environments. You'll notice that actions are a vector where each entry corresponds to the associated environment in the vector.

In A), we automatically reset the envs that have terminated so you can continue for an indefinite amount of steps. As environments don't have the same length, they stop at different times, this helps you reset terminated episodes on the fly.

Tips:
- The step_size return many observations, should you feed each one-by-one in your model, or the full step_size=10 one? The choice is yours!
- There exists multiple ways of exploring the dataset

In [57]:
for step in range(40):

    # A) Check if any environments terminated
    terminated_envs = env.get_terminated_env_indices()
    if terminated_envs:
        print(f"   ⚠️  Environments {terminated_envs} terminated")
        reset_obs, reset_infos = env.reset_specific_envs(terminated_envs)
        for i, env_id in enumerate(terminated_envs):
            infos[env_id] = reset_infos[i] # reset previous info dict

    # Generate random actions for all environments
    actions = np.random.randint(1, 3, size=env.num_envs)

    print(f"\n   Step {step + 1}:")
    print(f"      Actions: {actions}")

    # Take step
    observations, rewards, terminateds, truncateds, infos = env.step(actions)

    print(f"      Rewards: {rewards}")
    print(f"      Terminated: {terminateds}")
    print(f"      Obs: {observations[0].shape}") # observation shape of first environment
    print(f"      Active environments: {env.get_active_count()}/{env.num_envs}")

    # Show filtered info (production mode)
    for i, info in enumerate(infos):
        print(f"      Env {i} info: {info}")

env.close()


   Step 1:
      Actions: [1 1 2 1]


2026-02-18 14:59:03,271 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_step "HTTP/1.1 200 OK"


      Rewards: [-547.8807  -520.64185  140.74242 -524.24243]
      Terminated: [False False  True False]
      Obs: (10, 9)
      Active environments: 3/4
      Env 0 info: {'step': 10, 'episode_id': '35036e75-d5e6-4259-9ddf-436ff9a3f768', 'terminated': False, 'truncated': False}
      Env 1 info: {'step': 10, 'episode_id': '95213856-7fb8-45e1-81eb-e4e8623b23fb', 'terminated': False, 'truncated': False}
      Env 2 info: {'step': 1, 'episode_id': '3c4dd096-a106-4b0d-bae8-b87bb0a62a14', 'terminated': True, 'truncated': False}
      Env 3 info: {'step': 10, 'episode_id': '462b8bc4-c064-489d-a3eb-e622f7920b0e', 'terminated': False, 'truncated': False}
   ⚠️  Environments [2] terminated


2026-02-18 14:59:04,207 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"



   Step 2:
      Actions: [1 1 2 2]


...

2026-02-18 14:59:36,840 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_step "HTTP/1.1 200 OK"


      Rewards: [-547.6513   136.44388 -511.36447    0.     ]
      Terminated: [False  True False  True]
      Obs: (10, 9)
      Active environments: 2/4
      Env 0 info: {'step': 10, 'episode_id': '35036e75-d5e6-4259-9ddf-436ff9a3f768', 'terminated': False, 'truncated': False}
      Env 1 info: {'step': 1, 'episode_id': '95213856-7fb8-45e1-81eb-e4e8623b23fb', 'terminated': True, 'truncated': False}
      Env 2 info: {'step': 10, 'episode_id': '3c4dd096-a106-4b0d-bae8-b87bb0a62a14', 'terminated': False, 'truncated': False}
      Env 3 info: {'episode_id': '462b8bc4-c064-489d-a3eb-e622f7920b0e', 'terminated': True, 'truncated': False}
   ⚠️  Environments [1, 3] terminated


2026-02-18 14:59:38,763 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"



   Step 11:
      Actions: [2 1 2 1]


KeyboardInterrupt: 